In [2]:
#Load libraries 
import sspa
import networkx as nx
import pandas as pd
import numpy as np
import scipy
import matplotlib.pyplot as plt
import seaborn as sns

### Check sample overlap between datasets

In [2]:
df = pd.read_csv('Data/Su_COVID_metabolomics_processed.csv', index_col=0)
df2 = pd.read_csv('Data/Su_COVID_proteomics_processed.csv', index_col=0)

In [3]:
df.index= df.index.str.rstrip('-BL')  #remove 'BL' label from the cases (so I can match to proteomic data)

In [4]:
def sample_overlap(status):
    list1 = list(df[df['WHO_status']== status].index)
    list2 = list(df2[df2['WHO_status']== status].index)
    #print(list1)
    #print(list2)
    intersection = list(set(list1).intersection(list(set(list2))))  #set removes duplicates
    print(status, "      Common samples:",len(intersection),"         Metabolomic samples:",len(list1),"       Proteomic samples:",len(list2))
    #print(set(list1) - set(list2),set(list2) - set(list1))   #Different samples between the two

sample_overlap('0')
sample_overlap('1-2')
sample_overlap('3-4')
sample_overlap('5-7')

#print(df['WHO_status'].value_counts()) 

0       Common samples: 18          Metabolomic samples: 133        Proteomic samples: 123
1-2       Common samples: 45          Metabolomic samples: 45        Proteomic samples: 48
3-4       Common samples: 56          Metabolomic samples: 57        Proteomic samples: 59
5-7       Common samples: 27          Metabolomic samples: 28        Proteomic samples: 28


In [5]:
df.index= df.index.str.rstrip('-BL')  #remove 'BL' label from the cases (so I can match to proteomic data)

list1 = list(df.index)
list2 = list(df2.index)
#print(list1)
#print(list2)
intersection = list(set(list1).intersection(list(set(list2))))  #set removes duplicates

print(len(intersection))

146


### Obtaining pathway names for the graph

In [1]:
#Load dataset
metabolomic = nx.read_gml("../Cytoscape/metabolomic_prefiltered.gml")
proteomic = nx.read_gml("../Cytoscape/proteomic_prefiltered.gml")


NameError: name 'nx' is not defined

In [6]:
#Load dataset
#This is when the pathway scores are calculated subset to the 128 SAMPLES common to both datasets
metabolomic = nx.read_gml("../Cytoscape/Outdated/metabolomic_prefiltered_commoncases.gml")
proteomic = nx.read_gml("../Cytoscape/Outdated/proteomic_prefiltered_commoncases.gml")

#metabolomic = nx.read_gml("Cytoscape/metabolomic_prefiltered_mildcases.gml")
#proteomic = nx.read_gml("Cytoscape/proteomic_prefiltered_mildcases.gml")

#metabolomic = nx.read_gml("Cytoscape/metabolomic_prefiltered_severecases.gml")
#proteomic = nx.read_gml("Cytoscape/proteomic_prefiltered_severecases.gml")

In [7]:
print(proteomic.number_of_nodes())
print(proteomic.number_of_edges())

print(metabolomic.number_of_nodes())
print(metabolomic.number_of_edges())

#Pathways in common
common_nodes = list(set(proteomic.nodes).intersection(list(set(metabolomic.nodes))))
len(common_nodes)

578
166753
144
10296


47

In [8]:
#Filter out pathways that are not present in both omic sets

proteomic_copy =  proteomic.copy()
for n in list(proteomic.nodes()):
    if n not in common_nodes:
        proteomic_copy.remove_node(n)

metabolomic_copy =  metabolomic.copy()
for n in list(metabolomic.nodes()):
    if n not in common_nodes:
        metabolomic_copy.remove_node(n)



In [10]:
proteomic_copy.edges()

EdgeView([('R-HSA-112315', 'R-HSA-1428517'), ('R-HSA-112315', 'R-HSA-1483257'), ('R-HSA-112315', 'R-HSA-156580'), ('R-HSA-112315', 'R-HSA-15869'), ('R-HSA-112315', 'R-HSA-1592230'), ('R-HSA-112315', 'R-HSA-168249'), ('R-HSA-112315', 'R-HSA-196071'), ('R-HSA-112315', 'R-HSA-196849'), ('R-HSA-112315', 'R-HSA-196854'), ('R-HSA-112315', 'R-HSA-211859'), ('R-HSA-112315', 'R-HSA-211945'), ('R-HSA-112315', 'R-HSA-2142753'), ('R-HSA-112315', 'R-HSA-2151201'), ('R-HSA-112315', 'R-HSA-2187338'), ('R-HSA-112315', 'R-HSA-2262752'), ('R-HSA-112315', 'R-HSA-2408508'), ('R-HSA-112315', 'R-HSA-2408522'), ('R-HSA-112315', 'R-HSA-2980736'), ('R-HSA-112315', 'R-HSA-372790'), ('R-HSA-112315', 'R-HSA-373076'), ('R-HSA-112315', 'R-HSA-381340'), ('R-HSA-112315', 'R-HSA-388396'), ('R-HSA-112315', 'R-HSA-416476'), ('R-HSA-112315', 'R-HSA-418555'), ('R-HSA-112315', 'R-HSA-418594'), ('R-HSA-112315', 'R-HSA-428157'), ('R-HSA-112315', 'R-HSA-500792'), ('R-HSA-112315', 'R-HSA-556833'), ('R-HSA-112315', 'R-HSA-56632

### Plot the parameter values for both omics datasets 

In [7]:
def find_val(list1,list2,value):
    for node1, node2, data in proteomic_copy.edges(data=True):
        list1.append(data[value])
    for node1, node2, data in metabolomic_copy.edges(data=True):
        list2.append(data[value])



Comparing the Spearman p-values of the two omics:

In [8]:
proteomic_pval = []
metabolomic_pval = []

find_val(proteomic_pval,metabolomic_pval,'Spearman_pval')

#It's not possible to take the log10 of zero, we ignore the divide by zero warning
#Transform the p-values on a -log10 scale
#Any infinite values after the log-transform were originally zeroes


np.seterr(divide = 'ignore')    
 
log_proteomic_pval = -np.log10(proteomic_pval)    
log_metabolomic_pval = -np.log10(metabolomic_pval)

np.seterr(divide = 'warn') 
    

{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [ ]:
print(np.median(proteomic_pval))
print(np.median(metabolomic_pval))

print(np.mean(proteomic_pval))
print(np.mean(metabolomic_pval))

print(sum(i == 1  for i in metabolomic_pval))

In [ ]:
sns.scatterplot(x=log_proteomic_pval, y=log_metabolomic_pval,s=8)       #color='#FFD580',s=10
 #Some inf values that are not plotted (too far along axis)
                
plt.title('Comparison of Spearman correlation p-values',fontsize=16)
plt.xlabel("Proteomic Spearman p-values (-log10)",fontsize=13, labelpad=5) 
plt.ylabel("Metabolomic Spearman p-values (-log10)",fontsize=13, labelpad=10) 

#Changing the axis limits to focus on the significant area only
#plt.xlim(-np.log10(0.005),)   #-log_10(0.005) = 2.301...
#plt.ylim(-np.log10(0.005),)

#plt.savefig('Figures/omics_Spearman_pval.png' ,dpi=200,bbox_inches = 'tight' , pad_inches = 0.2 , facecolor='w')

Comparing the Spearman correlation coefficients of the two omics:

In [ ]:
proteomic_corr = []
metabolomic_corr = []
find_val(proteomic_corr,metabolomic_corr,'Squared_corr')


sns.scatterplot(x=proteomic_corr, y=metabolomic_corr,s=8)       #color='#FFD580',s=10
                
plt.title('Comparison of squared Spearman correlation coefficients',fontsize=16)
plt.xlabel("Proteomic Spearman correlation",fontsize=13, labelpad=5)
plt.ylabel("Metabolomic Spearman correlation",fontsize=13, labelpad=10) 

#plt.savefig('Figures/omics_spearman_corr.png' ,dpi=200,bbox_inches = 'tight' , pad_inches = 0.2 , facecolor='w')

In [ ]:
print(np.median(proteomic_corr))
print(np.median(metabolomic_corr))

print(np.mean(proteomic_corr))
print(np.mean(metabolomic_corr))

Highlight edges which are significant in Spearman correlation scatterplot:

In [ ]:
proteomic_copy.edges(data=True)

In [ ]:
proteomic_pval = []
metabolomic_pval = []
find_val(proteomic_pval,metabolomic_pval,'Spearman_pval')

proteomic_oc = []
metabolomic_oc = []
find_val(proteomic_oc,metabolomic_oc,'Overlap_coef')

In [ ]:
omics_corr_df = pd.DataFrame(
    {'Proteomic_corr': proteomic_corr,
     'Metabolomic_corr': metabolomic_corr,
     'Proteomic_pval': proteomic_pval,
     'Metabolomic_pval': metabolomic_pval,
     'Proteomic_overlap': proteomic_oc,
     "Metabolomic_overlap": metabolomic_oc
    })

In [ ]:
omics_corr_df.iloc[-10:,:]
omics_corr_df["Sig"] = np.nan
omics_corr_df["Overlap"] = np.nan
omics_corr_df


In [ ]:
for i in range(len(omics_corr_df)):
    if ((omics_corr_df.Proteomic_pval[i]) >= 0.005) and ((omics_corr_df.Metabolomic_pval[i]) < 0.005):
        omics_corr_df.Sig[i] = "Proteomic insignificant"
    elif ((omics_corr_df.Proteomic_pval[i]) < 0.005) and ((omics_corr_df.Metabolomic_pval[i]) >= 0.005):
        omics_corr_df.Sig[i] = "Metabolomic insignificant"
    elif ((omics_corr_df.Proteomic_pval[i]) >= 0.005) and ((omics_corr_df.Metabolomic_pval[i]) >= 0.005):
        omics_corr_df.Sig[i] = "Both insignificant"
    else:
        omics_corr_df.Sig[i] = "Significant"
        
display(omics_corr_df)


print(omics_corr_df.Sig.str.count("Significant").sum())

print(len(omics_corr_df[omics_corr_df['Metabolomic_corr'] == 0.9999999999999996]))  #31 out of 1,081 edges, 0.9999999999999996 for all cases, 1 for common cases subset
omics_corr_df['Metabolomic_corr'][1080]

In [ ]:
fig, ax = plt.subplots()  #i do this because ax.legend is needed to remove the legend title

sns.scatterplot(
    data=omics_corr_df,
    x="Proteomic_corr", 
    y="Metabolomic_corr",
    hue="Sig",
    hue_order = ['Metabolomic insignificant','Proteomic insignificant','Both insignificant','Significant'],
    #alpha=0.5,
    s=8)       #color='#FFD580',s=10
        
plt.title('Comparison of squared Spearman correlation coefficients',fontsize=16)
plt.xlabel("Proteomic Spearman correlation",fontsize=13, labelpad=5)
plt.ylabel("Metabolomic Spearman correlation",fontsize=13, labelpad=10) 

legend = ax.legend(bbox_to_anchor=(1.4,0.6), prop={'size': 8})
#plt.legend(framealpha=1, frameon = 'True', title="p-value",title_fontsize='large', prop={'size': 10}, bbox_to_anchor=(1.35, 0.7)) 

#plt.savefig('Figures/ALL_omics_spearman_corr_sigedges.png' ,dpi=200,bbox_inches = 'tight' , pad_inches = 0.2 , facecolor='w')

In [ ]:

for i in range(len(omics_corr_df)):
    if ((omics_corr_df.Proteomic_overlap[i]) < 0.5) and ((omics_corr_df.Metabolomic_overlap[i]) < 0.5):
        omics_corr_df.Overlap[i] = "Low overlap < 0.5"
    elif ((omics_corr_df.Proteomic_overlap[i]) >= 0.5) and ((omics_corr_df.Metabolomic_overlap[i]) < 0.5):
        omics_corr_df.Overlap[i] = "Proteomic overlap
    elif ((omics_corr_df.Proteomic_overlap[i]) < 0.5) and ((omics_corr_df.Metabolomic_overlap[i]) >= 0.5):
        omics_corr_df.Overlap[i] = "Metabolomic overlap"
    elif ((omics_corr_df.Proteomic_overlap[i]) >= 0.5) and ((omics_corr_df.Metabolomic_overlap[i]) >= 0.5):
        omics_corr_df.Overlap[i] = "Overlap for both"


display(omics_corr_df)


print(omics_corr_df.Overlap.str.count("Low overlap < 0.5").sum()) #766 out of 1,081 edges (same for whole dataset and also when subset to common cases)
print(len(omics_corr_df[omics_corr_df['Metabolomic_overlap'] == 0]))  #597 out of 1,081 edges (same for whole dataset and also when subset to common cases)

In [ ]:
fig, ax = plt.subplots()  #i do this because ax.legend is needed to remove the legend title

sns.scatterplot(
    data=omics_corr_df,
    x="Proteomic_corr", 
    y="Metabolomic_corr",
    hue="Overlap",
    hue_order = ['Proteomic overlap','Metabolomic overlap','Overlap for both','Low overlap < 0.5'],
    s=8)       #color='#FFD580',s=10
        
plt.title('Comparison of squared Spearman correlation coefficients',fontsize=16)
plt.xlabel("Proteomic Spearman correlation",fontsize=13, labelpad=5)
plt.ylabel("Metabolomic Spearman correlation",fontsize=13, labelpad=10) 

legend = ax.legend(bbox_to_anchor=(1.4,0.6), prop={'size': 8})
#plt.legend(framealpha=1, frameon = 'True', title="p-value",title_fontsize='large', prop={'size': 10}, bbox_to_anchor=(1.35, 0.7)) 

#plt.savefig('Figures/COMMONCCASES_omics_spearman_corr_oc.png' ,dpi=200,bbox_inches = 'tight' , pad_inches = 0.2 , facecolor='w')

Comparing the overlap coefficient of the two omics:

In [ ]:
proteomic_oc = []
metabolomic_oc = []

find_val(proteomic_oc,metabolomic_oc,'Overlap_coef')

sns.scatterplot(x=proteomic_oc, y=metabolomic_oc,s=8)       #color='#FFD580',s=10
                
plt.title('Comparison of overlap coefficients',fontsize=16)
plt.xlabel("Proteomic overlap coefficient",fontsize=13, labelpad=5)
plt.ylabel("Metabolomic overlap coefficient",fontsize=13, labelpad=10) 

#plt.savefig('Figures/omics_overlap_coef.png' ,dpi=200,bbox_inches = 'tight' , pad_inches = 0.2 , facecolor='w')

Since there are fewer points observed than expected, it is possible that many of the points are overlapping.
Therefore we should plot a regression line to check if there is a trend, and also to reduce the point transparency.

In [ ]:
#Obtain regression values
#https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html

plot = sns.regplot(x=proteomic_oc, y=metabolomic_oc,scatter_kws={'s':5,'alpha':0.3},line_kws={'linewidth':2})  #,color='red,x_jitter=0.005,y_jitter=0.005
plt.title('Comparison of overlap coefficients',fontsize=16)
plt.xlabel("Proteomic overlap coefficient",fontsize=13, labelpad=5)
plt.ylabel("Metabolomic overlap coefficient",fontsize=13, labelpad=10) 

res = scipy.stats.linregress(x=proteomic_oc, y=metabolomic_oc)
print(f"R-squared: {res.rvalue**2:.6f}")
plt.rcParams['font.size'] = 14  #change font size of the r-squared label
plt.text(0.5, 0.25, f"R-squared: {res.rvalue**2:.6f}")

#plt.savefig('Figures/omics_overlap_coef.png' ,dpi=200,bbox_inches = 'tight' , pad_inches = 0.2 , facecolor='w')


In [ ]:
print(np.median(proteomic_oc))
print(np.median(metabolomic_oc))

print(np.mean(proteomic_oc))
print(np.mean(metabolomic_oc))

print(sum(i == 0  for i in metabolomic_oc))
print(sum(i == 0  for i in proteomic_oc))

### Method 1 - Combine both pre-filtered networks together and combine the p-values

The networks were constructed in NetworkX but the edges have not been filtered out

In [ ]:
#https://networkx.org/documentation/stable/reference/algorithms/operators.html
# compose = "Compose graph G with H by combining nodes and edges into a single graph."

#If networks have the same edge, keep the attributes of the last network
#I.e. Right now the edges have the metabolite edge attributes
G = nx.compose(proteomic_copy,metabolomic_copy)

#len(proteomic_copy.edges & metabolomic_copy.edges)
#nx.draw(G, with_labels = True)

print(G.edges['R-HSA-112315', 'R-HSA-1483257'])

Combining the p-values

In [9]:
#Using Fisher's method to combine the p-values: Example code
p_values_list= [5.202380530055585e-20, 7.5913348658720385e-25]
print(scipy.stats.combine_pvalues(p_values_list, method='fisher'))  #output gives two numbers; first is the test statistic, second is the p-value (what we want)

#Using harmonic mean p-value method to combine the p-values: Example code
scipy.stats.hmean(p_values_list)


SignificanceResult(statistic=199.8804108441174, pvalue=3.986432815011612e-42)


1.5182448188841517e-24

In [13]:
edge_dict = {e: ([   proteomic_copy.edges[e]['Spearman_pval']  ,  metabolomic_copy.edges[e]['Spearman_pval']  ]) for e in proteomic_copy.edges & metabolomic_copy.edges}
print(edge_dict)

combined_pvals_fisher = []
combined_pvals_hmean = []

for k,v in edge_dict.items():
    
    #Fisher's method
    test_stat,pval = scipy.stats.combine_pvalues(v, method='fisher',weights=None)
    combined_pvals_fisher.append(pval)

    #Harmonic mean p-value
    combined_pvals_hmean.append(scipy.stats.hmean(v))

    

print(combined_pvals_fisher)
print(combined_pvals_hmean)

print(sum(i < 0.005 for i in combined_pvals_fisher))
print(sum(i < 0.005  for i in combined_pvals_hmean))

{('R-HSA-372790', 'R-HSA-418594'): [1.637881029477618e-144, 1.0], ('R-HSA-70326', 'R-HSA-9707564'): [3.9339571151363586e-10, 5.0246311975494385e-14], ('R-HSA-211859', 'R-HSA-9707564'): [2.7115235981050148e-27, 1.0], ('R-HSA-2262752', 'R-HSA-428157'): [0.0015854246060171254, 1.0], ('R-HSA-1483257', 'R-HSA-2980736'): [1.2315607045246778e-07, 0.05364154579219742], ('R-HSA-1483257', 'R-HSA-73884'): [4.206934424730389e-17, 4.980725088355118e-17], ('R-HSA-168249', 'R-HSA-196854'): [4.2019451503073545e-66, 1.0], ('R-HSA-73884', 'R-HSA-9707564'): [2.9070772023701827e-29, 1.0], ('R-HSA-416476', 'R-HSA-71387'): [2.3301084881861253e-12, 1.0], ('R-HSA-15869', 'R-HSA-428157'): [0.0003636496301264095, 6.552704004172038e-09], ('R-HSA-1592230', 'R-HSA-168249'): [2.00372016495351e-31, 1.0], ('R-HSA-2187338', 'R-HSA-5668914'): [1.6342464499210634e-24, 1.0], ('R-HSA-2142753', 'R-HSA-373076'): [1.5591264718108376e-37, 1.0], ('R-HSA-196854', 'R-HSA-5683826'): [1.8463101059649232e-30, 1.0], ('R-HSA-15869', 

c:\Users\jenna\anaconda3\envs\Imperial_Project2\Lib\site-packages\scipy\stats\_stats_py.py:9690: RuntimeWarning: divide by zero encountered in log
  statistic = -2 * np.sum(np.log(pvalues))


[1.0732405737001967e-25, 2.0528787859827566e-65]
[1.0, 1.0]
[2.4896279160987668e-05, 1.2294140167561724e-18]
[1.0038855237830238e-13, 1.0]
[6.017171449119095e-55, 1.0]
[0.00036923610193016364, 1.0]
[1.5469036169805635e-53, 1.0]
[1.1056512111158036e-28, 1.506259291187759e-05]
[1.0165039123793703e-23, 0.0016315602198733372]
[3.7825882550056194e-48, 1.0]
[1.0, 1.0]
[3.9240512811296525e-31, 1.0646579218102796e-08]
[4.548514892048173e-29, 1.0001196260559665e-26]
[7.922946041461978e-25, 1.0]
[7.737674623036185e-15, 6.472656324213871e-21]
[1.6888795920523326e-07, 0.0022367901259885757]
[9.310116348565803e-43, 1.0]
[2.2906753348909548e-89, 1.0]
[1.0, 1.0]
[1.4276351706510583e-19, 8.188087050602088e-17]
[4.0747719489446984e-33, 1.0]
[5.331673417166785e-32, 4.310576378587352e-90]
[4.901539778530262e-18, 0.016429099555032627]
[2.0233574706682637e-38, 4.310576378587352e-90]
[1.65735991924737e-81, 1.0]
[5.7607473329946976e-11, 1.0]
[1.0, 1.0]
[8.488125101524069e-05, 1.0]
[6.0325178642310645e-68, 1.

In [ ]:
#It's not possible to take the log10 of zero, we ignore the divide by zero warning
#The p-values of zero become infinity values instead (far along the axis, therefore are not plotted)
#Transform the p-values on a -log10 scale

np.seterr(divide = 'ignore')    

log_combined_pvals_fisher = -np.log10(combined_pvals_fisher)
log_combined_pvals_hmean = -np.log10(combined_pvals_hmean)

np.seterr(divide = 'warn') 

In [ ]:
len(combined_pvals_fisher)
len(combined_pvals_hmean)

In [ ]:
#Code aims to see whether the edges significant with the two methods are subsets of each other i.e. how many edges are significant for one but not the other

keylist = []
vallist = []
for i in range(1081):
    fisher = combined_pvals_fisher[i]
    hmean = combined_pvals_hmean[i]

    if fisher < 0.005 and hmean >= 0.005:
        vallist.append(hmean)
    if hmean < 0.005 and fisher >=0.005:
        keylist.append(fisher)
  

print(sum(i < 0.005 for i in combined_pvals_fisher))
print(sum(i < 0.005 for i in combined_pvals_hmean))

print(len(keylist))
print(len(vallist))

In [ ]:
sns.scatterplot(x=log_combined_pvals_fisher, y=log_combined_pvals_hmean,s=8)       #color='#FFD580',s=10
                
plt.title('Comparison of p-value combining methods',fontsize=16)
plt.xlabel("Fisher's method p-values (-log10)",fontsize=13, labelpad=5)
plt.ylabel("Harmonic mean p-values (-log10)",fontsize=13, labelpad=10) 

plt.xlim(0,)   #-log_10(0.005) = 2.301...
plt.ylim(0,)


#Changing the axis limits to focus on the boundary 
#plt.xlim(0,-np.log10(0.005)+1.5)   #-log_10(0.005) = 2.301...
#plt.ylim(0,-np.log10(0.005)+1.5)

#plt.axvline(x=-np.log10(0.005), color='r', linewidth=0.1, linestyle='-')
#plt.axhline(y=-np.log10(0.005), color='r', linewidth=0.1, linestyle='-')

#plt.savefig('Figures/p_val_combining_methods.png' ,dpi=200,bbox_inches = 'tight' , pad_inches = 0.2 , facecolor='w')

In [ ]:
#Using harmonic mean p-value method 
edge_data = {e:combined_pvals_hmean[index] for index,e in enumerate(proteomic_copy.edges & metabolomic_copy.edges) } # for e in proteomic_copy.edges & metabolomic_copy.edges
nx.set_edge_attributes(G, edge_data, 'Spearman_pval')

In [ ]:
#On the original single omic metabolomic and proteomic dataset showing the Spearman correlation, highlight which edges are significant after combining the p-values

def find_val(list1,list2,value):
    for node1, node2, data in proteomic_copy.edges(data=True):
        list1.append(data[value])
    for node1, node2, data in metabolomic_copy.edges(data=True):
        list2.append(data[value])

proteomic_corr = []
metabolomic_corr = []
find_val(proteomic_corr,metabolomic_corr,'Squared_corr')


combined_pval_list = []
for node1, node2, data in G.edges(data=True):
        combined_pval_list.append(data['Spearman_pval'])
 

omics_corr_df = pd.DataFrame(
    {'Proteomic_corr': proteomic_corr,
     'Metabolomic_corr': metabolomic_corr,
     'Combined_pval': combined_pval_list
    })

omics_corr_df.iloc[-10:,:]
omics_corr_df["Sig"] = np.nan

for i in range(len(omics_corr_df)):
    if ((omics_corr_df.Combined_pval[i]) >= 0.005):
        omics_corr_df.Sig[i] = "Insignificant"
    else:
        omics_corr_df.Sig[i] = "Significant"
        
display(omics_corr_df)


print(omics_corr_df.Sig.str.count("Significant").sum())  #936 edges significant for whole dataset, 585 edges for the common cases

fig, ax = plt.subplots()  #i do this because ax.legend is needed to remove the legend title

sns.scatterplot(
    data=omics_corr_df,
    x="Proteomic_corr", 
    y="Metabolomic_corr",
    hue="Sig",
    #alpha=0.5,
    s=8)       #color='#FFD580',s=10
        
plt.title('Comparison of squared Spearman correlation coefficients',fontsize=16)
plt.xlabel("Proteomic Spearman correlation",fontsize=13, labelpad=5)
plt.ylabel("Metabolomic Spearman correlation",fontsize=13, labelpad=10) 

legend = ax.legend(bbox_to_anchor=(1.26,0.6), prop={'size': 8})
#plt.legend(framealpha=1, frameon = 'True', title="p-value",title_fontsize='large', prop={'size': 10}, bbox_to_anchor=(1.35, 0.7)) 

#plt.savefig('Figures/ALL_COMBINEDPVAL_omics_spearman_corr_sigedges.png' ,dpi=200,bbox_inches = 'tight' , pad_inches = 0.2 , facecolor='w')

Comparing the average of the Spearman correlation coefficients with other methods i.e. harmonic mean, taking the max value of the Spearman results

In [ ]:
proteomic_copy.edges(data=True)

In [ ]:
edge_dict = {e: ([ proteomic_copy.edges[e]['Squared_corr']  ,  metabolomic_copy.edges[e]['Squared_corr']  ]) for e in proteomic_copy.edges & metabolomic_copy.edges}
print(edge_dict)

averaged_corr = []
combined_corr_hmean = []
max_corr = []

for k,v in edge_dict.items():

    #Taking the average of the two Spearman correlation coefficients
    averaged_corr.append((sum(v))/2)

    #Harmonic mean of the Spearman correlation coefficients
    combined_corr_hmean.append(scipy.stats.hmean(v))

    #Taking the maximum value from the two Spearman correlation coefficients
    max_corr.append(max(v))

print(averaged_corr)  
print(combined_corr_hmean)
print(max_corr)

In [ ]:
sns.scatterplot(x=averaged_corr, y=combined_corr_hmean,s=8)       #color='#FFD580',s=10
                
plt.title('Comparison of Spearman correlation combining methods',fontsize=16)
plt.xlabel("Average of correlation values",fontsize=13, labelpad=5)
plt.ylabel("Harmonic mean of correlation values",fontsize=13, labelpad=10) 

plt.xlim(0,)  
plt.ylim(0,)

#plt.savefig('Figures/COMMONCASES_correlation_combining_methods_hmean.png' ,dpi=200,bbox_inches = 'tight' , pad_inches = 0.2 , facecolor='w')

In [ ]:
sns.scatterplot(x=averaged_corr, y=max_corr,s=8)       #color='#FFD580',s=10
                
plt.title('Comparison of Spearman correlation combining methods',fontsize=16)
plt.xlabel("Average of correlation values",fontsize=13, labelpad=5)
plt.ylabel("Maximum of correlation values",fontsize=13, labelpad=10) 

plt.xlim(0,)  
plt.ylim(0,)

#plt.savefig('Figures/COMMONCASES_correlation_combining_methods_maxcorr.png' ,dpi=200,bbox_inches = 'tight' , pad_inches = 0.2 , facecolor='w')

In [ ]:
#Get the average for the Spearman correlation coefficients and the overlap coefficient
#HOWEVER it is not recommended to take the average of the p-values, thus we combine them using one of two methods: Fisher's method OR harmonic p-value

def add_attributes(attribute_name):
    edge_data = {e: ((proteomic_copy.edges[e][attribute_name] + metabolomic_copy.edges[e][attribute_name])/2) for e in proteomic_copy.edges & metabolomic_copy.edges}
    nx.set_edge_attributes(G, edge_data, attribute_name)


#Print all edge information (data=True means to display edge attribute data as well)
#print(G.edges(data=True))             

add_attributes("Squared_corr")
add_attributes("Overlap_coef")


print(proteomic_copy.edges['R-HSA-112315', 'R-HSA-1483257'])
print(metabolomic_copy.edges['R-HSA-112315', 'R-HSA-1483257'])
print(G.edges['R-HSA-112315', 'R-HSA-1483257'])

#print(proteomic_copy.edges['R-HSA-2262752', 'R-HSA-8978868'])
#print(metabolomic_copy.edges['R-HSA-2262752', 'R-HSA-8978868'])
#print(G.edges['R-HSA-2262752', 'R-HSA-8978868'])


In [ ]:
#Filter out edges based on Spearman p-value and Overlap Coefficient
edges_to_remove = [(u,v) for u,v,e in G.edges(data=True) if (e['Spearman_pval'] >= 0.005 or e['Overlap_coef'] >= 0.5)]
len(edges_to_remove)
edges_to_remove

In [ ]:
G.remove_edges_from(edges_to_remove)
print(G.number_of_nodes())
print(G.number_of_edges())
G.nodes()

In [ ]:
nx.draw(G, with_labels = True)

### Method 2 - Filter out edges separately by Spearman p-value then average out the Spearman correlation and Overlap Coefficient

In [ ]:
#Filter out edges based on Spearman p-value and Overlap Coefficient
edges_to_remove_proteomic = [(u,v) for u,v,e in proteomic_copy.edges(data=True) if (e['Spearman_pval'] >= 0.05 )]
print(len(edges_to_remove_proteomic))

edges_to_remove_metabolomic = [(u,v) for u,v,e in metabolomic_copy.edges(data=True) if (e['Spearman_pval'] >= 0.05 )]
print(len(edges_to_remove_metabolomic))

edges_to_remove = set( edges_to_remove_proteomic + edges_to_remove_metabolomic )
print(len(edges_to_remove))

G = nx.compose(proteomic_copy,metabolomic_copy)
print(G.edges['R-HSA-112315', 'R-HSA-1483257'])

def add_attributes(attribute_name):
    edge_data = {e: ((proteomic_copy.edges[e][attribute_name] + metabolomic_copy.edges[e][attribute_name])/2) for e in proteomic_copy.edges & metabolomic_copy.edges}
    nx.set_edge_attributes(G, edge_data, attribute_name)


#Print all edge information (data=True means to display edge attribute data as well)
#print(G.edges(data=True))             

add_attributes("Squared_corr")
add_attributes("Overlap_coef")


print(proteomic_copy.edges['R-HSA-112315', 'R-HSA-1483257'])
print(metabolomic_copy.edges['R-HSA-112315', 'R-HSA-1483257'])
print(G.edges['R-HSA-112315', 'R-HSA-1483257'])

G.remove_edges_from(edges_to_remove)
print(G.number_of_nodes())
print(G.number_of_edges())
G.nodes()

#Filter out edges based on Spearman p-value and Overlap Coefficient
edges_to_remove = [(u,v) for u,v,e in G.edges(data=True) if e['Overlap_coef'] >= 0.5]
len(edges_to_remove)

G.remove_edges_from(edges_to_remove)
print(G.number_of_nodes())
print(G.number_of_edges())
G.nodes()

### Setting node attributes (Next stage after running either Method 1 or Method 2)

In [ ]:
#Nodes with no edges
nodes_to_remove = list(nx.isolates(G))
nodes_to_remove

#['R-HSA-196071', 'R-HSA-2151201']

In [ ]:
#nodes_to_remove=['R-HSA-196071']
G.remove_nodes_from(nodes_to_remove)

In [ ]:
#IF YOU DON'T REMOVE THE EDGES, DON'T RUN THIS CODE

#Add the betweenness centrality as a node attribute
betweenness= nx.betweenness_centrality(G, normalized=True) #output as dictionary
#display(betweenness)
nx.set_node_attributes(G, betweenness, "betweenness")

#Add the betweenness centrality as a node attribute
degree= nx.degree_centrality(G) #output as dictionary
#display(degree)
nx.set_node_attributes(G, degree, "degrees")


#Check
#G.nodes['R-HSA-112315']#["betweenness"]
#G.nodes['R-HSA-196071']#["betweenness"]

In [ ]:
#Using Cecilia's code

#Creating graph
hierarchy = pd.read_csv('Data/ReactomePathwaysRelation.txt', sep='\t', header=None)

#From the pathways, subset to Homo sapiens only
hierarchy_hsa = hierarchy[hierarchy[0].str.contains('HSA')]

#Return unique values in the first column that is not in the second column as a numpy array
#These values are not child pathways in any instances
hierarchy_hsa_parents = np.setdiff1d(hierarchy_hsa[0], hierarchy_hsa[1])

#Add the unique values not in the second column as a second attached dataset to the bottom of the original data
#The first column represents the parent column, the second column is the child column
hierarchy_hsa_all = pd.concat([hierarchy_hsa, pd.DataFrame([hierarchy_hsa_parents, hierarchy_hsa_parents], index=[0, 1]).T])

#DiGraph is a directed graph
H = nx.from_pandas_edgelist(hierarchy_hsa, source=0, target=1, create_using=nx.DiGraph())


In [ ]:
#Find the root pathway 

def find_root(H,child):
    #Find parent from child 
    parent = list(H.predecessors(child))

    #Keep the loop going until the highest level is reached
    if len(parent) == 0:
        return child
    else:  
        return find_root(H, parent[0])

hierarchy_hsa_all['Root'] = [find_root(H, i) for i in hierarchy_hsa_all[1]]

hierarchy_hsa_all.columns = ['Parent', 'Child', 'Root']

#There are 83 instances of duplicates, however all the child duplicates have the same root (even though different parents) after checking



#Convert pathway ID to name
root_path = pd.read_excel('Data/Root_pathways.xlsx', header=None)
root_pathway_dict = {root_path[0][i]:root_path[1][i] for i in range(0,len(root_path))}



#Assign name to each node
root_pathways = {}
for pathway in list(H.nodes):
    index = hierarchy_hsa_all.Child[hierarchy_hsa_all.Child == pathway].index.tolist()[0]
    root_pathway  = hierarchy_hsa_all.Root[index]
    label = root_pathway_dict[root_pathway]
    root_pathways[pathway] = label




In [ ]:
#shows all the root pathways in Reactome
set(hierarchy_hsa_all['Root'] )
#shows all the root pathways present in the original dataset
set(root_pathways.values())

nx.set_node_attributes(G, root_pathways, "root_pathway")

print(G.number_of_nodes()) 

In [ ]:

louvain_clusters = nx.community.louvain_communities(G, weight='Squared_corr',seed=100,resolution=1.5)
print(len(louvain_clusters))

louvain_dict = {}
for index,grouping in enumerate(louvain_clusters):
    for pathway in grouping:
        louvain_dict[pathway] = index+1

nx.set_node_attributes(G, louvain_dict, "louvain")
    

Note that for the separate omics analyses I use different pathway files for the separate metabolomics and proteomics data (since different compounds form the pathways), however since we take the intersection of the compounds for data integration it is fine to use the metabolomics pathways file to load in the pathway names.

In [ ]:
reactome_pathways = sspa.process_gmt("Data/Reactome_Homo_sapiens_pathways_compounds_R84.gmt")

pathway_name_dict = {reactome_pathways.index[i]:reactome_pathways["Pathway_name"][i] for i in range(0,len(reactome_pathways))}
pathway_name_dict['R-HSA-1483257']
pathway_name_dict = {k:pathway_name_dict[k] for k in list(G.nodes)}
#G = nx.relabel_nodes(G, pathway_name_dict)
nx.set_node_attributes(G, pathway_name_dict, "pathway_name")
G.nodes

In [ ]:
print(G.number_of_nodes())
print(G.number_of_edges())
G.edges['R-HSA-112315', 'R-HSA-1483257']#["Spearman_pval"]
G.nodes['R-HSA-112315']#["betweenness"]

In [ ]:
#nx.write_gml(G,'Cytoscape/integrated_commoncases_harmonic.gml')
#nx.write_gml(G,'Cytoscape/integrated_all_harmonic.gml')
#nx.write_gml(G,'Cytoscape/integrated_mildcases_harmonic.gml')
#nx.write_gml(G,'Cytoscape/integrated_severecases_harmonic.gml')